In [1]:
import re
import os
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
from netCDF4 import Dataset
import geopandas as gpd
from shapely.geometry import Point
import pandas as pd
import calendar
from datetime import datetime, timedelta

### Year 2017, Year 2018, Year 2019 need to be regenerated

In [2]:
nc_file_path = '/global/scratch/users/liuwenjin021011/data/CHIRPS/chirps-v2.0.2017.days_p05.nc'
lat_min, lat_max = 18, 54
lon_min, lon_max = 73, 135

# Open the NetCDF file
ds = xr.open_dataset(nc_file_path)

# Filter the dataset for the latitude and longitude range of China
ds_china = ds.sel(latitude=slice(lat_min, lat_max), longitude=slice(lon_min, lon_max))
ds_china

<xarray.Dataset>
Dimensions:    (latitude: 640, longitude: 1240, time: 365)
Coordinates:
  * latitude   (latitude) float32 18.02 18.07 18.12 18.17 ... 49.87 49.92 49.97
  * longitude  (longitude) float32 73.02 73.07 73.12 73.18 ... 134.9 134.9 135.0
  * time       (time) datetime64[ns] 2017-01-01 2017-01-02 ... 2017-12-31
Data variables:
    precip     (time, latitude, longitude) float32 ...
Attributes: (12/15)
    Conventions:       CF-1.6
    title:             CHIRPS Version 2.0
    history:           created by Climate Hazards Group
    version:           Version 2.0
    date_created:      2018-01-17
    creator_name:      Pete Peterson
    ...                ...
    reference:         Funk, C.C., Peterson, P.J., Landsfeld, M.F., Pedreros,...
    comments:           time variable denotes the first day of the given day....
    acknowledgements:  The Climate Hazards Group InfraRed Precipitation with ...
    ftp_url:           ftp://chg-ftpout.geog.ucsb.edu/pub/org/chg/products/CH...
    website:           http://chg.geog.ucsb.edu/data/chirps/index.html
    faq:               http://chg-wiki.geog.ucsb.edu/wiki/CHIRPS_FAQ

In [3]:
# get the info of precip, NOTICE that precip is recored in mm/day, but the criteria for wet is 1cm!!!

def get_variable_info(dataset, variable_name):
    
    if variable_name in dataset.variables:
        variable = dataset[variable_name]
        unit = variable.attrs.get('units', 'No units attribute')
        description = variable.attrs.get('long_name', 'No description attribute')
        return unit, description
    else:
        return 'Variable not found in dataset', ''

unit, description = get_variable_info(ds_china, 'precip')
print(f"Unit: {unit}\nDescription: {description}")

Unit: mm/day
Description: Climate Hazards group InfraRed Precipitation with Stations


In [4]:
import xarray as xr
import pandas as pd

def process_precipitation_data(ds_china):
    # Calculate the rolling sum of the previous 14 days
    precip_rolling_sum = ds_china['precip'].rolling(time=14, min_periods=1).sum()

    # Convert to DataFrame for easier manipulation
    df = ds_china['precip'].to_dataframe(name='precipitation')
    df['sum_precip_14_days'] = precip_rolling_sum.to_dataframe(name='sum_precip_14_days')

    # The 'precipitation' column already contains the daily precipitation values

    return df

# Define the function to determine status
def determine_status(row):
    if row['sum_precip_14_days'] > 10:
        return 'wet'
    else:
        return 'dry'


processed_df = process_precipitation_data(ds_china)
processed_df = processed_df.reset_index()
# processed_df['status'] = processed_df.apply(determine_status, axis=1)

In [8]:
processed_df

,time,latitude,longitude,precipitation,sum_precip_14_days
0,2017-01-01,18.024994,73.024994,0.0,0.000000
1,2017-01-01,18.024994,73.074997,0.0,0.000000
2,2017-01-01,18.024994,73.125000,0.0,0.000000
3,2017-01-01,18.024994,73.175003,0.0,0.000000
4,2017-01-01,18.024994,73.224991,0.0,0.000000
...,...,...,...,...,...
289663995,2017-12-31,49.974998,134.774994,0.0,1.578361
289663996,2017-12-31,49.974998,134.825012,0.0,1.429319
289663997,2017-12-31,49.974998,134.875000,0.0,1.547112
289663998,2017-12-31,49.974998,134.924988,0.0,1.586476


In [9]:
processed_df.to_csv('/global/scratch/users/liuwenjin021011/data/CHIRPS/FactorA_Processed_Precip_Year_2017.csv', header = True, index = False)

In [ ]:
processed_df['status'] = processed_df.apply(determine_status, axis=1)
processed_df